# 🌍 World Wide Web Crawler

Crawler yang bisa menjelajahi internet secara luas dengan **otomatis mencari SEED URLs**.

### Cara Kerja:
1. Input kata kunci
2. Crawler otomatis mencari titik awal dari berbagai sumber
3. Crawl dan ikuti link secara rekursif
4. Simpan URL yang kontennya mengandung kata kunci

## 📦 Step 1: Install Library

In [ ]:
!pip install requests beautifulsoup4 -q
print("✅ Library berhasil diinstall!")

✅ Library berhasil diinstall!


## ⚙️ Step 2: Import Library

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, quote_plus
from collections import deque
from datetime import datetime
import time
import re
import random

print("✅ Library berhasil diimport!")

✅ Library berhasil diimport!


## 🌍 Step 3: World Wide Web Crawler Class

In [ ]:
class WorldWideCrawler:
    """
    Web Crawler yang bisa menjelajahi internet secara luas.
    Otomatis mencari SEED URLs dari berbagai sumber.
    """

    def __init__(self, keywords, max_results=10, max_pages=100, max_depth=3, delay=1.0, verbose=True):
        """
        Args:
            keywords (str): Kata kunci yang dicari (pisah dengan koma)
            max_results (int): Maksimal URL hasil yang match dengan kata kunci
            max_pages (int): Maksimal halaman yang di-crawl (limit untuk tidak infinite)
            max_depth (int): Kedalaman crawling
            delay (float): Jeda antar request
            verbose (bool): Tampilkan log detail
        """
        # Parse keywords
        if isinstance(keywords, str):
            self.keywords = [kw.strip().lower() for kw in keywords.split(',') if kw.strip()]
        else:
            self.keywords = [kw.lower() for kw in keywords]

        self.max_results = max_results
        self.max_pages = max_pages
        self.max_depth = max_depth
        self.delay = delay
        self.verbose = verbose

        # Storage
        self.visited_urls = set()
        self.matched_urls = []
        self.queue = deque()
        self.discovered_domains = set()

        # Stats
        self.stats = {
            'pages_crawled': 0,
            'pages_matched': 0,
            'domains_discovered': 0,
            'errors': 0,
            'start_time': None,
            'end_time': None
        }

        # User agents untuk rotasi
        self.user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        ]

    def log(self, message, level="INFO"):
        """Log dengan timestamp."""
        if self.verbose:
            timestamp = datetime.now().strftime("%H:%M:%S")
            emoji = {
                "INFO": "ℹ️", "SUCCESS": "✅", "ERROR": "❌",
                "WARN": "⚠️", "DEBUG": "🔧", "FOUND": "🎯",
                "CRAWL": "🕷️", "SEED": "🌱", "DOMAIN": "🌐"
            }
            print(f"   {emoji.get(level, 'ℹ️')} [{timestamp}] {message}")

    def get_headers(self):
        """Generate random headers."""
        return {
            'User-Agent': random.choice(self.user_agents),
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
        }

    def discover_seed_urls(self):
        """
        Otomatis mencari SEED URLs dari berbagai sumber berdasarkan kata kunci.
        """
        print(f"\n{'─' * 70}")
        print("🌱 FASE 1: MENCARI SEED URLs OTOMATIS")
        print(f"{'─' * 70}")

        seed_urls = set()
        keyword_query = ' '.join(self.keywords)
        encoded_query = quote_plus(keyword_query)

        # Sumber-sumber untuk mendapatkan seed URLs
        sources = [
            # 1. Wikipedia - hub dengan banyak external links
            {
                'name': 'Wikipedia Search',
                'url': f'https://en.wikipedia.org/w/index.php?search={encoded_query}',
                'selector': 'a',
                'filter': lambda u: u.startswith('http') and 'wikipedia' not in u
            },
            # 2. Wikipedia Indonesia
            {
                'name': 'Wikipedia ID',
                'url': f'https://id.wikipedia.org/w/index.php?search={encoded_query}',
                'selector': 'a',
                'filter': lambda u: u.startswith('http') and 'wikipedia' not in u
            },
            # 3. DuckDuckGo HTML
            {
                'name': 'DuckDuckGo',
                'url': f'https://html.duckduckgo.com/html/?q={encoded_query}',
                'selector': 'a.result__a',
                'filter': lambda u: u.startswith('http')
            },
            # 4. Bing
            {
                'name': 'Bing',
                'url': f'https://www.bing.com/search?q={encoded_query}',
                'selector': 'li.b_algo a',
                'filter': lambda u: u.startswith('http') and 'bing.com' not in u and 'microsoft.com' not in u
            },
            # 5. GitHub - untuk topik programming
            {
                'name': 'GitHub',
                'url': f'https://github.com/search?q={encoded_query}&type=repositories',
                'selector': 'a',
                'filter': lambda u: u.startswith('http') and 'github.com' not in u
            },
            # 6. Reddit
            {
                'name': 'Reddit Search',
                'url': f'https://www.reddit.com/search/?q={encoded_query}',
                'selector': 'a',
                'filter': lambda u: u.startswith('http') and 'reddit.com' not in u
            },
        ]

        # Hub websites Indonesia (untuk kata kunci Indonesia)
        indonesian_hubs = [
            'https://www.kompas.com/',
            'https://www.detik.com/',
            'https://www.liputan6.com/',
            'https://www.tribunnews.com/',
            'https://kumparan.com/',
            'https://www.cnnindonesia.com/',
            'https://tekno.kompas.com/',
            'https://inet.detik.com/',
        ]

        # Programming/Tech hubs
        tech_hubs = [
            'https://dev.to/',
            'https://medium.com/',
            'https://stackoverflow.com/',
            'https://www.freecodecamp.org/',
            'https://www.geeksforgeeks.org/',
            'https://realpython.com/',
            'https://www.w3schools.com/',
            'https://www.tutorialspoint.com/',
            'https://www.petanikode.com/',
            'https://www.dicoding.com/',
            'https://www.codepolitan.com/',
        ]

        # Tambahkan hub websites sebagai seed
        self.log("Menambahkan Hub Websites...", "SEED")
        for hub in indonesian_hubs + tech_hubs:
            seed_urls.add(hub)
            domain = urlparse(hub).netloc
            self.discovered_domains.add(domain)
        self.log(f"Ditambahkan {len(indonesian_hubs) + len(tech_hubs)} hub websites", "SUCCESS")

        # Coba setiap sumber
        for source in sources:
            self.log(f"Mencari dari {source['name']}...", "SEED")

            try:
                response = requests.get(
                    source['url'],
                    headers=self.get_headers(),
                    timeout=10,
                    allow_redirects=True
                )

                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # Cari links
                    links = soup.select(source['selector'])
                    count = 0

                    for link in links:
                        href = link.get('href', '')

                        # Untuk DuckDuckGo, URL ada di href langsung
                        if href and source['filter'](href):
                            # Bersihkan URL
                            clean_url = href.split('?')[0] if '?' in href and 'uddg=' not in href else href

                            # Ekstrak domain
                            try:
                                domain = urlparse(clean_url).netloc
                                if domain and domain not in self.discovered_domains:
                                    seed_urls.add(clean_url)
                                    self.discovered_domains.add(domain)
                                    count += 1

                                    if count <= 3:  # Log beberapa contoh
                                        self.log(f"  + {domain}", "DOMAIN")
                            except:
                                pass

                    self.log(f"Ditemukan {count} domain baru dari {source['name']}", "SUCCESS")
                else:
                    self.log(f"{source['name']}: Status {response.status_code}", "WARN")

            except Exception as e:
                self.log(f"{source['name']}: {type(e).__name__}", "ERROR")

            time.sleep(1)  # Delay antar sumber

        self.stats['domains_discovered'] = len(self.discovered_domains)

        print(f"\n   📊 Total SEED URLs: {len(seed_urls)}")
        print(f"   🌐 Total Domains: {len(self.discovered_domains)}")

        return list(seed_urls)

    def is_valid_url(self, url):
        """Cek apakah URL valid untuk di-crawl."""
        try:
            parsed = urlparse(url)

            if parsed.scheme not in ['http', 'https']:
                return False

            # Skip file types
            skip_ext = ['.pdf', '.jpg', '.jpeg', '.png', '.gif', '.mp4', '.mp3',
                       '.zip', '.rar', '.exe', '.doc', '.docx', '.xls', '.ppt']
            if any(parsed.path.lower().endswith(ext) for ext in skip_ext):
                return False

            # Skip social media (terlalu banyak noise)
            skip_domains = [
                'facebook.com', 'twitter.com', 'instagram.com', 'tiktok.com',
                'linkedin.com', 'pinterest.com', 'tumblr.com',
                'google.com', 'bing.com', 'yahoo.com', 'duckduckgo.com',
                'amazon.com', 'ebay.com', 'aliexpress.com',
                'play.google.com', 'apps.apple.com'
            ]
            if any(domain in parsed.netloc for domain in skip_domains):
                return False

            return True
        except:
            return False

    def extract_text(self, soup):
        """Ekstrak teks dari HTML."""
        for tag in soup(['script', 'style', 'nav', 'footer', 'header', 'aside', 'noscript']):
            tag.decompose()

        text = soup.get_text(separator=' ', strip=True)
        text = re.sub(r'\s+', ' ', text).lower()
        return text

    def check_keywords(self, text):
        """Cek apakah teks mengandung kata kunci."""
        matched = [kw for kw in self.keywords if kw in text]
        return len(matched) > 0, matched

    def extract_links(self, soup, base_url):
        """Ekstrak links dari halaman."""
        links = []
        base_domain = urlparse(base_url).netloc

        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            full_url = urljoin(base_url, href)

            # Bersihkan URL
            parsed = urlparse(full_url)
            clean_url = f"{parsed.scheme}://{parsed.netloc}{parsed.path}".rstrip('/')

            if self.is_valid_url(clean_url) and clean_url not in self.visited_urls:
                links.append(clean_url)

                # Track domain baru
                domain = parsed.netloc
                if domain not in self.discovered_domains:
                    self.discovered_domains.add(domain)
                    self.stats['domains_discovered'] = len(self.discovered_domains)

        return list(set(links))

    def crawl_url(self, url, depth):
        """Crawl satu URL."""
        if url in self.visited_urls:
            return []

        if len(self.matched_urls) >= self.max_results:
            return []

        if self.stats['pages_crawled'] >= self.max_pages:
            return []

        self.visited_urls.add(url)
        self.stats['pages_crawled'] += 1

        domain = urlparse(url).netloc
        self.log(f"[{self.stats['pages_crawled']}/{self.max_pages}] Depth:{depth} | {domain}", "CRAWL")

        try:
            response = requests.get(url, headers=self.get_headers(), timeout=10)

            if response.status_code != 200:
                self.log(f"Status {response.status_code}", "WARN")
                return []

            # Cek content type
            content_type = response.headers.get('content-type', '')
            if 'text/html' not in content_type:
                return []

            soup = BeautifulSoup(response.text, 'html.parser')
            text = self.extract_text(soup)

            # Cek kata kunci
            found, matched_keywords = self.check_keywords(text)

            if found:
                self.stats['pages_matched'] += 1

                title = soup.title.string if soup.title else "No Title"
                title = title.strip()[:80] if title else "No Title"

                result = {
                    'url': url,
                    'title': title,
                    'domain': domain,
                    'matched_keywords': matched_keywords,
                    'depth': depth
                }
                self.matched_urls.append(result)

                self.log(f"🎯 MATCH! [{self.stats['pages_matched']}/{self.max_results}] {title[:40]}...", "FOUND")
                self.log(f"   Keywords: {matched_keywords}", "SUCCESS")

            # Ekstrak links jika belum mencapai max depth
            if depth < self.max_depth:
                return self.extract_links(soup, url)

            return []

        except requests.exceptions.Timeout:
            self.stats['errors'] += 1
            return []
        except Exception as e:
            self.stats['errors'] += 1
            return []

    def crawl(self, custom_seeds=None):
        """
        Mulai crawling.

        Args:
            custom_seeds (list): Optional - custom seed URLs. Jika None, akan auto-discover.
        """
        print("\n" + "=" * 70)
        print("🌍 WORLD WIDE WEB CRAWLER")
        print("=" * 70)
        print(f"🔑 Kata Kunci     : {', '.join(self.keywords)}")
        print(f"🎯 Max Hasil      : {self.max_results}")
        print(f"📄 Max Pages      : {self.max_pages}")
        print(f"📏 Max Depth      : {self.max_depth}")
        print(f"⏱️  Delay          : {self.delay}s")
        print("=" * 70)

        self.stats['start_time'] = datetime.now()

        # Fase 1: Discover seed URLs
        if custom_seeds:
            seed_urls = custom_seeds
            print(f"\n🌱 Menggunakan {len(seed_urls)} custom seed URLs")
        else:
            seed_urls = self.discover_seed_urls()

        # Shuffle untuk variasi
        random.shuffle(seed_urls)

        # Fase 2: Mulai crawling
        print(f"\n{'─' * 70}")
        print("🕷️  FASE 2: CRAWLING")
        print(f"{'─' * 70}")

        # Masukkan seed ke queue
        for url in seed_urls:
            if self.is_valid_url(url):
                self.queue.append((url, 0))

        # BFS Crawling
        while self.queue:
            # Cek stopping conditions
            if len(self.matched_urls) >= self.max_results:
                self.log(f"Mencapai max hasil ({self.max_results})", "INFO")
                break

            if self.stats['pages_crawled'] >= self.max_pages:
                self.log(f"Mencapai max pages ({self.max_pages})", "INFO")
                break

            url, depth = self.queue.popleft()

            if url in self.visited_urls:
                continue

            # Crawl
            new_links = self.crawl_url(url, depth)

            # Tambahkan links baru (batasi per halaman)
            for link in new_links[:15]:
                if link not in self.visited_urls:
                    self.queue.append((link, depth + 1))

            # Progress setiap 20 halaman
            if self.stats['pages_crawled'] % 20 == 0:
                print(f"\n   📊 Progress: {self.stats['pages_crawled']} crawled | "
                      f"{self.stats['pages_matched']} matched | "
                      f"{len(self.discovered_domains)} domains | "
                      f"{len(self.queue)} in queue")

            # Delay
            time.sleep(self.delay)

        self.stats['end_time'] = datetime.now()

        return self.matched_urls

    def display_results(self):
        """Tampilkan hasil."""
        duration = (self.stats['end_time'] - self.stats['start_time']).total_seconds()

        print("\n" + "=" * 70)
        print("📋 HASIL CRAWLING")
        print("=" * 70)
        print(f"🔑 Kata Kunci         : {', '.join(self.keywords)}")
        print(f"🕷️  Halaman Di-crawl   : {self.stats['pages_crawled']}")
        print(f"🎯 Halaman Match      : {self.stats['pages_matched']}")
        print(f"🌐 Domain Ditemukan   : {self.stats['domains_discovered']}")
        print(f"❌ Errors             : {self.stats['errors']}")
        print(f"⏱️  Durasi             : {duration:.1f} detik")
        print("-" * 70)

        if self.matched_urls:
            print("\n📄 URL YANG MENGANDUNG KATA KUNCI:\n")
            for i, item in enumerate(self.matched_urls, 1):
                print(f"[{i}] {item['title']}")
                print(f"    🔗 {item['url']}")
                print(f"    🔑 Keywords: {', '.join(item['matched_keywords'])}")
                print(f"    🌐 Domain: {item['domain']} | Depth: {item['depth']}")
                print()
        else:
            print("\n⚠️ Tidak ada halaman yang mengandung kata kunci.")
            print("\n💡 Tips:")
            print("   - Gunakan kata kunci lebih umum")
            print("   - Naikkan max_pages")
            print("   - Naikkan max_depth")

        print("\n" + "=" * 70)

    def save_results(self, filename=None):
        """Simpan hasil ke file."""
        if not self.matched_urls:
            print("⚠️ Tidak ada hasil.")
            return None

        if not filename:
            ts = datetime.now().strftime("%Y%m%d_%H%M%S")
            kw = '_'.join(self.keywords)[:15]
            filename = f"worldwide_crawl_{kw}_{ts}.txt"

        with open(filename, 'w', encoding='utf-8') as f:
            f.write("WORLD WIDE WEB CRAWLER - HASIL\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Kata Kunci      : {', '.join(self.keywords)}\n")
            f.write(f"Halaman Crawled : {self.stats['pages_crawled']}\n")
            f.write(f"Halaman Match   : {self.stats['pages_matched']}\n")
            f.write(f"Domains         : {self.stats['domains_discovered']}\n")
            f.write(f"Waktu           : {datetime.now()}\n")
            f.write("\n" + "-" * 50 + "\n\n")

            for i, item in enumerate(self.matched_urls, 1):
                f.write(f"[{i}] {item['title']}\n")
                f.write(f"    URL: {item['url']}\n")
                f.write(f"    Keywords: {', '.join(item['matched_keywords'])}\n")
                f.write(f"    Domain: {item['domain']}\n\n")

        print(f"\n💾 Hasil disimpan: {filename}")
        return filename

print("✅ WorldWideCrawler siap!")

✅ WorldWideCrawler siap!


## 🚀 Step 4: Jalankan Crawler

In [ ]:
# ============================================================
# 🔧 KONFIGURASI
# ============================================================

# Kata kunci yang dicari dalam KONTEN halaman
KATA_KUNCI = "python, pemrograman"

# Maksimal halaman yang MATCH dengan kata kunci
MAX_HASIL = 10

# Maksimal halaman yang di-crawl (limit agar tidak infinite)
MAX_PAGES = 200

# Kedalaman crawling
MAX_DEPTH = 3

# Jeda antar request (detik)
DELAY = 0.5

# Tampilkan log detail
VERBOSE = True

# ============================================================

In [ ]:
# Jalankan!
crawler = WorldWideCrawler(
    keywords=KATA_KUNCI,
    max_results=MAX_HASIL,
    max_pages=MAX_PAGES,
    max_depth=MAX_DEPTH,
    delay=DELAY,
    verbose=VERBOSE
)

# Crawl! (seed URLs akan dicari otomatis)
results = crawler.crawl()

# Tampilkan hasil
crawler.display_results()

# Simpan
crawler.save_results()


🌍 WORLD WIDE WEB CRAWLER
🔑 Kata Kunci     : python, pemrograman
🎯 Max Hasil      : 10
📄 Max Pages      : 200
📏 Max Depth      : 3
⏱️  Delay          : 0.5s

──────────────────────────────────────────────────────────────────────
🌱 FASE 1: MENCARI SEED URLs OTOMATIS
──────────────────────────────────────────────────────────────────────
   🌱 [06:19:35] Menambahkan Hub Websites...
   ✅ [06:19:35] Ditambahkan 19 hub websites
   🌱 [06:19:35] Mencari dari Wikipedia Search...
   🌐 [06:19:35]   + foundation.wikimedia.org
   🌐 [06:19:35]   + developer.wikimedia.org
   🌐 [06:19:35]   + www.wikimedia.org
   ✅ [06:19:35] Ditemukan 4 domain baru dari Wikipedia Search
   🌱 [06:19:36] Mencari dari Wikipedia ID...
   🌐 [06:19:37]   + id.wikibooks.org
   ✅ [06:19:37] Ditemukan 1 domain baru dari Wikipedia ID
   🌱 [06:19:38] Mencari dari DuckDuckGo...
   ⚠️ [06:19:38] DuckDuckGo: Status 202
   🌱 [06:19:39] Mencari dari Bing...
   ✅ [06:19:39] Ditemukan 0 domain baru dari Bing
   🌱 [06:19:40] Mencari dar

'worldwide_crawl_python_pemrogra_20260130_062026.txt'

## 📥 Step 5: Download Hasil

In [ ]:
try:
    from google.colab import files
    import glob
    hasil = glob.glob("worldwide_crawl_*.txt")
    if hasil:
        files.download(hasil[-1])
except:
    print("File di direktori lokal")

---

## 📖 Penjelasan

### Cara Kerja:

```
┌─────────────────────────────────────────────────────────┐
│  FASE 1: AUTO-DISCOVER SEED URLs                        │
│  ─────────────────────────────────────────────────────  │
│  • Wikipedia (ID & EN)                                  │
│  • DuckDuckGo                                           │
│  • Bing                                                 │
│  • GitHub                                               │
│  • Reddit                                               │
│  • Hub websites (Kompas, Detik, Dev.to, dll)           │
└─────────────────────────┬───────────────────────────────┘
                          │
                          ▼
┌─────────────────────────────────────────────────────────┐
│  FASE 2: CRAWLING                                       │
│  ─────────────────────────────────────────────────────  │
│  1. Kunjungi halaman                                    │
│  2. Baca konten                                         │
│  3. Cek kata kunci ──────► MATCH? ──► Simpan!          │
│  4. Ekstrak links                                       │
│  5. Ikuti links (BFS)                                   │
│  6. Ulangi sampai max_results atau max_pages           │
└─────────────────────────────────────────────────────────┘
```

### Parameter:

| Parameter | Fungsi |
|-----------|--------|
| `keywords` | Kata kunci dicari di dalam konten |
| `max_results` | Stop setelah N halaman match |
| `max_pages` | Limit total halaman yang di-crawl |
| `max_depth` | Seberapa dalam ikuti link |

### Tips:
- Kata kunci **umum** ("python") → lebih banyak hasil
- Kata kunci **spesifik** → lebih sedikit tapi relevan
- Naikkan `max_pages` untuk jangkauan lebih luas
- Naikkan `max_depth` untuk crawl lebih dalam